In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr
import pandas as pd
import os
from tqdm import tqdm, tnrange, tqdm_notebook

DF = pd.DataFrame
arr = np.array

# 读取数据集

In [2]:
dirPath = "E:\\Code\\_largeData\\Github--Open-Course-Learning--A04\\Project\\multiclass classification\\data preprocessed\\tf-idf"


trainSet = np.loadtxt(dirPath + '\\train.csv', delimiter=",")
trainSetLabel = np.loadtxt(dirPath + '\\train_label.csv', delimiter=",")

validateSet = np.loadtxt(dirPath + '\\validate.csv', delimiter=",")
validateSetLabel = np.loadtxt(dirPath + '\\validate_label.csv', delimiter=",")

testSet = np.loadtxt(dirPath + '\\test.csv', delimiter=",")

trainSet.shape, validateSet.shape, testSet.shape

((43766, 1644), (18756, 1644), (8671, 1644))

# KNN

In [3]:
def knn_classify(dataSet, labels, k, inputVector):
    '''使用knn对输入的向量进行分类，使用欧式距离'''
    #得到输入向量与数据集的差值的平方
    diffMat = (dataSet - inputVector)**2
    #计算每一行上元素的和
    distancesSquare = diffMat.sum(axis=1)
    #可以不用计算距离，用距离的平方也可以，这样效率更高
    #print(distancesSquare)
    #得到 K 个近邻的下标
    kNeighborsIndex  = distancesSquare.argpartition(k-1)[0:k]
    
    #计算 K 个近邻的距离倒数
    kDatas = 1/distancesSquare[kNeighborsIndex]
    #得到 对应的 K 个标签
    kLabels = labels[kNeighborsIndex]
    
    predictLabel = np.NAN
    maxDis = -np.inf
    for i in np.unique(kLabels):
        dis = np.sum(kDatas[kLabels==i]) 
        #print(dis)
        if dis > maxDis:
            maxDis = dis
            predictLabel = i
    #返回分类结果
    return predictLabel

##############测试程序#######################
knn_classify(arr([[1,1,0,0],
              [0,1,1,1],
              [1,0,0,1]]), arr([1, 0, 1]), 3, arr([0,1,1,0]))
##############测试程序#######################

0

# 计算平均准确率

In [4]:
def calcAvg(predict, actual):
    all_labels = list(set(actual))
    tot = 0.0
    for i in all_labels:
        counter = Counter(predict[actual == i] == i)
        hit_num, miss_num = counter[True], counter[False]
        tot += hit_num / (hit_num + miss_num)
    return tot/len(all_labels)

calcAvg(arr([1,2,3]), arr([3,2,1]))

0.3333333333333333

In [5]:
trainSet[0:5,:]

array([[ 0.01835862,  0.01180725,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03481808,  0.02239306,  0.03657256, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.01828628, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.03935749,  0.03213952, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.01515149, ...,  0.        ,
         0.        ,  0.        ]])

In [6]:
def run_knn_classify(dataSet, k):
    ansLabel = []
    for i in tnrange(len(dataSet), leave=False):
        ansLabel.append(knn_classify(trainSet, trainSetLabel, k, dataSet[i]))
    return arr(ansLabel)

def getTestAvg(k):
    ansLabel = run_knn_classify(validateSet, k)
    return calcAvg(ansLabel, validateSetLabel)

In [7]:
ansLabel = run_knn_classify(validateSet, 1)
calcAvg(ansLabel, validateSetLabel)

0.3726901463845958

In [9]:
Counter(trainSetLabel)
Counter(validateSetLabel)

Counter({0.0: 13606, 1.0: 18255, 2.0: 11905})

Counter({0.0: 5692, 1.0: 7802, 2.0: 5262})

In [8]:
Counter(ansLabel)

Counter({0.0: 13186, 1.0: 3174, 2.0: 2396})